# Coordinate Calculator and Label Compiler

Inputs:
- location in pixels of beginning and end of axes
- range of axes
- units of axis range
- user's desired spacing for x coordinates
- NO legend input! how to specify different colors/markers that separates the curves (for later stages)
    - could eventually have mask look for the legend (distinctive enough) to learn what types of distinguishing features exist between the lines on the graph
    - I will ignore this part for now
- detectron2 output:
    - list of dicts, one dict for each image
    - each dict has field 'panoptic_seg' which refers to a (named?) tuple.
    - the first field in the tuple is a tensor with the ID of each pixel
    - the second field is a list of dicts with one dict per ID.
    - entries in this dict are 'id' which is the ID that matches the first field tensor, and 'category_id'
    
Outputs:
- dictionary with the following fields:
    - coordinates: a dictionary, with fields equal to the curve labels (one for each label), and entries that are a list of tuples, each with (x, and y coordinates in same units as input image)
    - units: (a tuple of x and y units)
    - beginning and end of curve
    - NO labels (for later stages): a list of labels for each curve, taken from legend if possible, or automatic (data1, data2, etc)


Options for coordinates:
1. based on the actual axis, choose a set of x coordinates for which we want to know the y coordinates (for each curve)
2. based on pixels, grab points that occur every "few" pixels along the x axis, then convert to coordinates

- could do the first option by finding out how many pixels are between each of the given x points, and using that number of pixels paired with the second option.
- do we want user to input the spacing between points, or should we use a default value that seems reasonable based on the range of the x axis?
- should the different curves have uniform x axis values (so y values are directly comparable), or should we have the sets of data points for each curve start at the 'beginning' of the curve and end at the 'end'


Outlier cases:
- nested plots
- weird legend formatting

In [2]:
# uniform x axis but also save the very first and last endpoints of the curve 

### Notes, TODO
- adjust the x and y pixel locations to lie on the same horizontal and vertical lines
- put the coordinate and pixel min max values in a dictionary or something more compact

In [3]:
# create the axis info dictionary
def get_axis_info(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax, ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax):
    # if the input from kev is in a more compact form extract this info below before passing into the scale funcitons
    x_scale = get_x_scale(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax)
    y_scale = get_y_scale(ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax)
    pixel_origin = (xpixelmin, ypixelmax) # assumes that y pixel max is the larger y value
    axis_info_dict = {'pixel_origin': pixel_origin,
                      'x_scale': x_scale,
                      'y_scale': y_scale}
    return axis_info_dict

In [1]:
# establish scaling from pixel to real units
def get_x_scale(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax):
    # the x pixel and x coordinate count up in the same direction
    pixel_range = xpixelmax - xpixelmin
    coordinate_range = xcoordinatemax - xcoordinatemin
    x_scale = pixel_range / coordinate_range
    return x_scale # pixels per coordinate


def get_y_scale(ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax):
    # y pixel count down and y coordinate count up from origin
    pixel_range = ypixelmin - ypixelmax
    coordinate_range = ycoordinatemax - ycoordinatemin
    y_scale = pixel_range / coordinate_range
    return y_scale # pixels per coordinate

In [ ]:
# convert pixel location to coordinates
def pixel_to_coords(pixel_loc, axis_info_dict):
    # pixel_loc is a tuple (x,y) of pixel location starting from top left
    
    # get signed distance from pixel to origin in x and y(pixel units):
    pixel_distance_x = pixel_loc[0] - axis_info_dict['pixel_origin'][0]
    pixel_distance_y = axis_info_dict['pixel_origin'][1] - pixel_loc[1]
    
    # pixels / (pixel/coord) = coord
    coord_x = pixel_distance_x / axis_info_dict['x_scale']
    coord_y = pixel_distance_y / axis_info_dict['y_scale']
    return (coord_x, coord_y)

In [4]:
# each curve is one ID. get all pixel locations for one ID
def get_pixels_for_id(ID, pixel_tensor):
    pixel_array = np.array(pixel_tensor)
    result = np.where(pixel_array == ID)
    pixel_lst = list(zip(result[0], result[1]))
    return pixel_lst

maybe incorporate later:

convert ID to the label from the legend input

def get_label_for_id():
    return label

In [ ]:
# pretty much call the avg y pixel function for each unique x pixel value
def clean_pixel_lst(pixel_lst):
    return cleaned_pixel_lst

In [ ]:
# average all y pixels for a given x pixel
def get_avg_y_pixel():
    return avg_y_pixel

In [6]:
# TODO add in the cleaning of the coordinates based on y averaging
# and x axis unification
# maybe break this into two functions. first creates a pixel dict

# create coordinate dictionary to add to the output dict
def create_pixel_dict(panoptic_seg):
    # initialize dict
    pixel_dict = {}
    
    # get list of IDs:
    lst_of_dicts = panoptic_seg[1]
    
    for id_dict in lst_of_dicts:
        # get ID:
        ID = id_dict['id']
        pixel_lst = get_pixels_for_id(ID)
        cleaned_pixel_lst = clean_pixel_lst(pixel_lst)
        
        # add the list of pixels for this ID to the pixel dict
        pixel_dict[str(ID)] = cleaned_pixel_lst
    return pixel_dict

In [7]:
# TODO add in the cleaning of the coordinates based on y averaging
# and x axis unification
# maybe break this into two functions. first creates a pixel dict

# create coordinate dictionary to add to the output dict
def create_coordinate_dict(pixel_dict, axis_info_dict):
    # initialize dict
    coordinate_dict = {}
    
    for ID in pixel_dict.keys():
        pixel_lst = pixel_dict[ID]
        
        # convert pixels to coordinates
        coordinate_lst = []
        for pixel_loc in pixel_lst:
            coordinate_lst.append(pixel_to_coords(pixel_loc, axis_info_dict))
        
        # add the list of coordinates for this ID to the coordinate dict
        coordinate_dict[str(ID)] = coordinate_lst
    return coordinate_dict

In [ ]:
def get_start_end(pixel_dict):
    return

In [8]:
# create the output dictionary and input the units of axis and labels directly from the input info,
# as well as the coordinate dictionary
def create_output_dict(panoptic_seg, axis_info_dict, units):
    pixel_dict = create_pixel_dict(panoptic_seg)
    output_dict = {}
    output_dict['coordinates'] = create_coordinate_dict(pixel_dict, axis_info_dict)
    output_dict['start_end'] = get_start_end(pixel_dict)
    output_dict['units'] = units
    return output_dict